In [ ]:
import sys

!{sys.executable} -m pip install ijson tabulate

In [ ]:
import hpq
import html
import IPython.display
import ijson
import json
import tabulate


def impl(connection, query):
    objs = []
    e = None
    try:
        for obj in ijson.items(connection.stream(query), "item"):
            objs.append(obj)
    except (hpq.WebSocketClient.MidStreamError, hpq.WebSocketClient.RejectError) as ex:
        e = ex
    if e is None:
        IPython.display.display(IPython.display.HTML("<h1>200 OK</h1>"))
    else:
        http_status_code = html.escape(str(e.json["http_status_code"]))
        reason = html.escape(e.json["reason"])
        IPython.display.display(
            IPython.display.HTML(f"<h1>{http_status_code} {reason}</h1>")
        )
        message = html.escape(e.json["message"])
        IPython.display.display(IPython.display.HTML(f"<p>{message}</p>"))
        if isinstance(e, hpq.WebSocketClient.RejectError):
            IPython.display.display(
                IPython.display.HTML(f"<p>Request was rejected</p>")
            )
        else:
            IPython.display.display(
                IPython.display.HTML(f"<p>Failure reported mid-response</p>")
            )
            raw = html.escape(json.dumps(e.accepted))
            IPython.display.display(IPython.display.HTML(f"<pre>{raw}</pre>"))
        raw = html.escape(str(e))
        IPython.display.display(IPython.display.HTML(f"<pre>{raw}</pre>"))
    if len(objs) == 0:
        IPython.display.display(IPython.display.HTML(f"<p>No results</p>"))
    else:
        table = tabulate.tabulate(
            map(hpq.format, objs), tablefmt="html", headers="keys"
        )
        IPython.display.display(IPython.display.HTML(table))


def take(num, iter):
    count = 0
    for obj in iter:
        if count == num:
            return
        yield obj
        count += 1

In [ ]:
import hpq

connection = hpq.create_web_socket_client()
impl(
    connection,
    {
        "query": "error",
        "source": "cme",
        "http_status_code": 404,
        "message": "HPQ error handling demonstration",
    },
)

In [ ]:
import hpq
import ijson
import json

connection = hpq.create_web_socket_client()
impl(
    connection, {"query": "error", "source": "cme", "body": "", "http_status_code": 500}
)
body = json.dumps(
    list(
        take(
            10,
            ijson.items(
                connection.stream(
                    {
                        "query": "ticks",
                        "source": "cme",
                        "product": "NGX2",
                        "date": "2022-09-22",
                        "start_time": "22:00:00",
                        "end_time": "22:15:00",
                        "time_zone": "UTC",
                        "messages": ["TRD"],
                    }
                ),
                "item",
            ),
        )
    )
)
connection.cancel()
body = body[: len(body) - 1] + ","
impl(
    connection,
    {"query": "error", "source": "cme", "body": body, "http_status_code": 500},
)